# Note since model building is required so i can't use twitter data since that is customer support data
# I can use a api (say azure language studio) to get a target column customer support size is too much and it won't make sense
# This is a testing Notebook

In [8]:
import numpy as np
import pandas as pd

In [11]:
#Data Import
data=pd.read_csv("C:/Users/aditya/Desktop/2024/GENAI_Assignments/Assignment -2/NLP_PLATFORM/data/train.csv" ,encoding="ISO-8859-1")
data=data[['text','sentiment']]
data.head()


,text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD I will miss you here in San Diego!!!,negative
2,my boss is bullying me...,negative
3,what interview! leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...",negative


In [12]:
data['sentiment'].unique()

array(['neutral', 'negative', 'positive'], dtype=object)

# Data Cleaning

In [60]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

# Data Import
data = pd.read_csv("C:/Users/aditya/Desktop/2024/GENAI_Assignments/Assignment -2/NLP_PLATFORM/data/train.csv", encoding="ISO-8859-1")
data = data[['text', 'sentiment']]

# Convert non-string values to empty strings
data['text'] = data['text'].fillna('')

# Convert text to lowercase
data['text'] = data['text'].str.lower()

# Remove HTML tags
data['text'] = data['text'].apply(lambda x: re.sub(r'<.*?>', '', x))

# Remove special characters and numbers
data['text'] = data['text'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))

# Remove punctuation
data['text'] = data['text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

# Remove URLs
data['text'] = data['text'].apply(lambda x: re.sub(r'http\S+|www\S+|https\S+', '', x, flags=re.MULTILINE))

# Remove stopwords
stop_words = set(stopwords.words('english'))
data['text'] = data['text'].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word not in stop_words]))

# Perform stemming
stemmer = PorterStemmer()
data['text'] = data['text'].apply(lambda x: ' '.join([stemmer.stem(word) for word in word_tokenize(x)]))

# Handle white spaces
data['text'] = data['text'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

print("Preprocessing done")


Preprocessing done


In [61]:
data.head(5)

,text,sentiment
0,id respond go,neutral
1,sooo sad miss san diego,negative
2,boss bulli,negative
3,interview leav alon,negative
4,son couldnt put releas alreadi bought,negative


In [52]:
data.head()

,text,sentiment
0,id respond go,neutral
1,sooo sad miss san diego,negative
2,boss bulli,negative
3,interview leav alon,negative
4,son couldnt put releas alreadi bought,negative


In [53]:
import spacy
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [55]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aditya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [57]:
# Tokenize the corpus
tokenized_corpus = [word_tokenize(doc.lower()) for doc in data['text']]

# Train Word2Vec model
model = Word2Vec(sentences=tokenized_corpus, size=100, window=5, min_count=1, workers=4)

# Save the trained model
model.save("custom_word2vec.model")

In [62]:
import pandas as pd
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np

# Load the trained Word2Vec model
model = Word2Vec.load("custom_word2vec.model")



# Extract text and labels
X_text = data['text']
y = data['sentiment']

# Tokenize and compute average word vectors for each document
X = []
for text in X_text:
    tokens = word_tokenize(text.lower())
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    if vectors:
        doc_vector = np.mean(vectors, axis=0)
        X.append(doc_vector)
    else:
        X.append(np.zeros(model.vector_size))  # Use zero vector for out-of-vocabulary words

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train SVM classifier
svm_classifier = SVC(kernel='linear')  # You can experiment with different kernels
svm_classifier.fit(X_train, y_train)

# Make predictions
y_pred = svm_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.5160996907404038
